In [1]:
import pandas as pd


shadow_data = pd.read_csv('data/loan_data.csv')
target_data = pd.read_csv('data/credit_risk_dataset.csv')

In [2]:
cols = set(target_data.columns)- set(shadow_data.columns)
target_data = target_data.drop(cols, axis = 1)

cols = set(shadow_data.columns) - set(target_data.columns) 
shadow_data = shadow_data.drop(cols, axis = 1)

In [3]:
cols_to_encode = ['person_home_ownership', 'loan_intent']

shadow_data_enc = pd.get_dummies(shadow_data, columns= cols_to_encode)
target_data_enc = pd.get_dummies(target_data, columns= cols_to_encode)

In [4]:
shadow_data_enc

,person_age,person_income,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,22.0,71948.0,35000.0,16.02,0.49,3.0,1,False,False,False,True,False,False,False,False,True,False
1,21.0,12282.0,1000.0,11.14,0.08,2.0,0,False,False,True,False,False,True,False,False,False,False
2,25.0,12438.0,5500.0,12.87,0.44,3.0,1,True,False,False,False,False,False,False,True,False,False
3,23.0,79753.0,35000.0,15.23,0.44,2.0,1,False,False,False,True,False,False,False,True,False,False
4,24.0,66135.0,35000.0,14.27,0.53,4.0,1,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,47971.0,15000.0,15.66,0.31,3.0,1,False,False,False,True,False,False,False,True,False,False
44996,37.0,65800.0,9000.0,14.07,0.14,11.0,1,False,False,False,True,False,False,True,False,False,False
44997,33.0,56942.0,2771.0,10.02,0.05,10.0,1,False,False,False,True,True,False,False,False,False,False
44998,29.0,33164.0,12000.0,13.23,0.36,6.0,1,False,False,False,True,False,True,False,False,False,False


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC, LinearSVC

n_shadow = 5

n = len(shadow_data_enc)//n_shadow

shadow_X = shadow_data_enc.drop('loan_status', axis = 1)
shadow_y = shadow_data_enc['loan_status']

shadow_df = pd.DataFrame()

shadow_out = []

for i in range(n_shadow):

    X = shadow_X[(n*i):(n*(i+1))]
    y = shadow_y[(n*i):(n*(i+1))]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

    shadow_out += list(X_test.index)

    shadow_model = RandomForestClassifier()
    shadow_model.fit(X_train, y_train)

    pred_train = shadow_model.predict(X_train)
    pred_test = shadow_model.predict(X_test)

    acc_train = accuracy_score(y_train, pred_train)
    acc_test = accuracy_score(y_test, pred_test)

    print("Accuracy Train:", acc_train)
    print("Accuracy Test:", acc_test)

    preds_shadow = shadow_model.predict_proba(X)

    temp_df = pd.DataFrame({
        'prediction': preds_shadow.tolist(),
        'in_or_out': [1] * len(X)
    })

    shadow_df = pd.concat([shadow_df, temp_df])


shadow_df.loc[shadow_df.index.isin(shadow_out), 'in_or_out'] = 0


Accuracy Train: 1.0
Accuracy Test: 0.9068888888888889
Accuracy Train: 1.0
Accuracy Test: 0.91
Accuracy Train: 1.0
Accuracy Test: 0.8953333333333333
Accuracy Train: 1.0
Accuracy Test: 0.9366666666666666
Accuracy Train: 1.0
Accuracy Test: 0.8824444444444445


In [40]:
shadow_df

,prediction,in_or_out
0,"[0.01, 0.99]",1
1,"[0.83, 0.17]",1
2,"[0.01, 0.99]",1
3,"[0.0, 1.0]",1
4,"[0.05, 0.95]",0
...,...,...
8995,"[0.0, 1.0]",1
8996,"[0.21, 0.79]",1
8997,"[0.34, 0.66]",1
8998,"[0.01, 0.99]",1


In [8]:
shadow_df[['prediction_0', 'prediction_1']] = pd.DataFrame(shadow_df['prediction'].tolist(), index=shadow_df.index)

shadow_df = shadow_df.drop('prediction', axis=1)

shadow_df

,in_or_out,prediction_0,prediction_1
0,1,0.01,0.99
1,1,0.83,0.17
2,1,0.01,0.99
3,1,0.00,1.00
4,0,0.05,0.95
...,...,...,...
8995,1,0.00,1.00
8996,1,0.21,0.79
8997,1,0.34,0.66
8998,1,0.01,0.99


In [13]:
X = shadow_df.drop('in_or_out', axis = 1)
y = shadow_df['in_or_out']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

attack_model = RandomForestClassifier(n_estimators=100)

attack_model.fit(X_train, y_train)
pred_train = attack_model.predict(X_train)
pred_test = attack_model.predict(X_test)

acc_train = accuracy_score(y_train, pred_train)
acc_test = accuracy_score(y_test, pred_test)

print("Accuracy Train:", acc_train)
print("Accuracy Test:", acc_test)

Accuracy Train: 0.5338666666666667
Accuracy Test: 0.524


CIFAR-10

In [58]:
# 5. Definición de la red neuronal
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import random
import numpy as np

batch_size = 32

# 3. Transformaciones para preprocesamiento
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                              download=True, transform=transform)
full_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                             download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [60]:
train_indices = list(range(len(full_trainset)))

random.shuffle(train_indices)
shadow_train_indices = [train_indices[i:i + 10000] for i in range(0, 50000, 10000)]

# Crear conjuntos de entrenamiento y evaluación de 5000 ejemplos para cada shadow model
shadow_datasets = []
for indices in shadow_train_indices:
    shadow_train = indices[:5000]
    shadow_eval = indices[5000:]
    shadow_datasets.append((torch.utils.data.Subset(full_trainset, shadow_train),
                            torch.utils.data.Subset(full_trainset, shadow_eval)))

In [61]:
test_indices = list(range(len(full_testset)))
random.shuffle(test_indices)
target_train_indices = test_indices[:5000]
target_eval_indices = test_indices[5000:10000]

target_trainset = torch.utils.data.Subset(full_testset, target_train_indices)
target_evalset = torch.utils.data.Subset(full_testset, target_eval_indices)

# DataLoaders para el target model
target_trainloader = torch.utils.data.DataLoader(target_trainset, batch_size=batch_size, shuffle=True)
target_evalloader = torch.utils.data.DataLoader(target_evalset, batch_size=batch_size, shuffle=False)

In [64]:
from collections import Counter
for i, indices in enumerate(shadow_train_indices):
    labels = [full_trainset.targets[idx] for idx in indices]
    class_counts = Counter(labels)
    total = sum(class_counts.values())
    print(f"\nProporciones de clase en la partición {i + 1}:")
    for class_id, count in class_counts.items():
        print(f"{classes[class_id]}: {count / total:.2%} ({count}/{total})")


Proporciones de clase en la partición 1:
cat: 9.87% (987/10000)
bird: 9.88% (988/10000)
plane: 9.94% (994/10000)
car: 10.11% (1011/10000)
deer: 10.15% (1015/10000)
horse: 9.73% (973/10000)
truck: 10.34% (1034/10000)
dog: 10.53% (1053/10000)
frog: 9.65% (965/10000)
ship: 9.80% (980/10000)

Proporciones de clase en la partición 2:
bird: 9.78% (978/10000)
cat: 10.30% (1030/10000)
dog: 9.29% (929/10000)
truck: 10.15% (1015/10000)
horse: 10.48% (1048/10000)
ship: 10.05% (1005/10000)
car: 9.74% (974/10000)
deer: 10.48% (1048/10000)
frog: 10.07% (1007/10000)
plane: 9.66% (966/10000)

Proporciones de clase en la partición 3:
plane: 10.13% (1013/10000)
frog: 10.13% (1013/10000)
truck: 9.44% (944/10000)
car: 9.95% (995/10000)
deer: 9.72% (972/10000)
bird: 10.44% (1044/10000)
cat: 10.21% (1021/10000)
ship: 9.65% (965/10000)
horse: 10.14% (1014/10000)
dog: 10.19% (1019/10000)

Proporciones de clase en la partición 4:
dog: 10.14% (1014/10000)
car: 10.30% (1030/10000)
cat: 9.73% (973/10000)
frog: 1

In [62]:
def train(net, trainloader, criterion, optimizer, epochs):
    net.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{epochs}] - Pérdida: {running_loss / len(trainloader):.3f}")
    print("Finalizó el entrenamiento")


def get_criterion_optimizer(model, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return criterion, optimizer

# 8. Función de evaluación
def get_predictions(net, dataloader):
    net.eval()
    all_preds = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = net(inputs)
            preds = outputs.cpu().numpy()
            all_preds.extend(preds)
    return np.array(all_preds)


In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo usado: {device}")

epochs = 50
learning_rate = 0.001
num_shadow_models = 5

all_shadow_data = []

for i in range(num_shadow_models):
    print(f"\nEntrenando Shadow Model {i + 1}/{num_shadow_models}")
    shadow_net = Net().to(device)
    criterion, optimizer = get_criterion_optimizer(shadow_net, learning_rate)
    
    shadow_trainloader = torch.utils.data.DataLoader(shadow_datasets[i][0], batch_size=batch_size, shuffle=True)
    shadow_evalloader = torch.utils.data.DataLoader(shadow_datasets[i][1], batch_size=batch_size, shuffle=False)
    
    train(shadow_net, shadow_trainloader, criterion, optimizer, epochs)
    
    # Guardar predicciones en el conjunto de entrenamiento y evaluación
    shadow_train_preds = get_predictions(shadow_net, shadow_trainloader)
    shadow_eval_preds = get_predictions(shadow_net, shadow_evalloader)

    df_train = pd.DataFrame(shadow_train_preds)
    df_train['membership'] = 1
    df_train['model'] = f'shadow_{i+1}'

    df_eval = pd.DataFrame(shadow_eval_preds)
    df_eval['membership'] = 0
    df_eval['model'] = f'shadow_{i+1}'

    all_shadow_data.append(df_train)
    all_shadow_data.append(df_eval)

final_df = pd.concat(all_shadow_data, ignore_index=True)
final_df.to_csv("shadow_models_predictions.csv", index=False)


Dispositivo usado: cuda

Entrenando Shadow Model 1/5
Epoch [1/50] - Pérdida: 1.974
Epoch [2/50] - Pérdida: 1.600
Epoch [3/50] - Pérdida: 1.453
Epoch [4/50] - Pérdida: 1.332
Epoch [5/50] - Pérdida: 1.217
Epoch [6/50] - Pérdida: 1.108
Epoch [7/50] - Pérdida: 1.009
Epoch [8/50] - Pérdida: 0.907
Epoch [9/50] - Pérdida: 0.812
Epoch [10/50] - Pérdida: 0.731
Epoch [11/50] - Pérdida: 0.679
Epoch [12/50] - Pérdida: 0.578
Epoch [13/50] - Pérdida: 0.495
Epoch [14/50] - Pérdida: 0.445
Epoch [15/50] - Pérdida: 0.393
Epoch [16/50] - Pérdida: 0.353
Epoch [17/50] - Pérdida: 0.308
Epoch [18/50] - Pérdida: 0.303
Epoch [19/50] - Pérdida: 0.259
Epoch [20/50] - Pérdida: 0.222
Epoch [21/50] - Pérdida: 0.209
Epoch [22/50] - Pérdida: 0.184
Epoch [23/50] - Pérdida: 0.168
Epoch [24/50] - Pérdida: 0.168
Epoch [25/50] - Pérdida: 0.156
Epoch [26/50] - Pérdida: 0.161
Epoch [27/50] - Pérdida: 0.145
Epoch [28/50] - Pérdida: 0.139
Epoch [29/50] - Pérdida: 0.134
Epoch [30/50] - Pérdida: 0.109
Epoch [31/50] - Pérdida: 0

In [70]:
print("\nEntrenando Target Model")
target_net = Net().to(device)
criterion, optimizer = get_criterion_optimizer(target_net, learning_rate)
train(target_net, target_trainloader, criterion, optimizer, epochs)

# Obtener predicciones del target model en el conjunto de entrenamiento y evaluación
target_train_preds = get_predictions(target_net, target_trainloader)
target_eval_preds = get_predictions(target_net, target_evalloader)

# Crear DataFrames para el target model
df_target_train = pd.DataFrame(target_train_preds)
df_target_train['membership'] = 1
df_target_train['model'] = 'target'

df_target_eval = pd.DataFrame(target_eval_preds)
df_target_eval['membership'] = 0
df_target_eval['model'] = 'target'

# Combinar y guardar el DataFrame del target model
target_df = pd.concat([df_target_train, df_target_eval], ignore_index=True)
target_df.to_csv("target_model_predictions.csv", index=False)


Entrenando Target Model
Epoch [1/50] - Pérdida: 1.943
Epoch [2/50] - Pérdida: 1.594
Epoch [3/50] - Pérdida: 1.438
Epoch [4/50] - Pérdida: 1.319
Epoch [5/50] - Pérdida: 1.193
Epoch [6/50] - Pérdida: 1.087
Epoch [7/50] - Pérdida: 0.992
Epoch [8/50] - Pérdida: 0.897
Epoch [9/50] - Pérdida: 0.809
Epoch [10/50] - Pérdida: 0.718
Epoch [11/50] - Pérdida: 0.630
Epoch [12/50] - Pérdida: 0.568
Epoch [13/50] - Pérdida: 0.514
Epoch [14/50] - Pérdida: 0.440
Epoch [15/50] - Pérdida: 0.372
Epoch [16/50] - Pérdida: 0.333
Epoch [17/50] - Pérdida: 0.316
Epoch [18/50] - Pérdida: 0.268
Epoch [19/50] - Pérdida: 0.252
Epoch [20/50] - Pérdida: 0.237
Epoch [21/50] - Pérdida: 0.202
Epoch [22/50] - Pérdida: 0.198
Epoch [23/50] - Pérdida: 0.183
Epoch [24/50] - Pérdida: 0.170
Epoch [25/50] - Pérdida: 0.170
Epoch [26/50] - Pérdida: 0.145
Epoch [27/50] - Pérdida: 0.134
Epoch [28/50] - Pérdida: 0.149
Epoch [29/50] - Pérdida: 0.148
Epoch [30/50] - Pérdida: 0.142
Epoch [31/50] - Pérdida: 0.130
Epoch [32/50] - Pérdida

In [66]:
preds_df = pd.read_csv('shadow_models_predictions.csv')
preds_df

,0,1,2,3,4,5,6,7,8,9,membership,model
0,-1.652007,-6.284151,-9.003187,-18.519178,-14.634975,-13.900465,-12.657706,-12.385738,11.159037,1.992938,1,shadow_1
1,-2.655606,-4.602125,-33.881447,-1.052191,-29.675043,-22.137732,-25.530573,-17.036850,12.059818,18.106739,1,shadow_1
2,-17.018867,-14.634188,-10.818435,3.325087,-16.855425,-4.700167,16.599197,-11.875827,-24.686830,0.170351,1,shadow_1
3,-9.245818,-20.693743,4.279681,12.980601,-9.917517,-7.856993,-3.544590,-16.709167,-11.583319,-12.016417,1,shadow_1
4,7.121928,22.040407,-31.075886,-18.922323,-34.153347,-34.467167,-29.663330,-25.651325,0.913894,3.945587,1,shadow_1
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-16.954830,-6.548839,-17.009138,-2.095353,4.266451,-5.086629,17.508175,-10.753840,-30.206448,-16.472815,0,shadow_5
49996,-0.142852,18.894518,-28.627031,-14.916573,-20.597652,-16.409283,-12.104305,-27.935184,9.934397,-10.556874,0,shadow_5
49997,4.462792,-17.379757,-10.379684,-13.213210,9.778001,-19.039822,-27.522718,16.050873,-19.092741,-11.387939,0,shadow_5
49998,-9.664123,-12.527952,-6.688738,7.011596,1.357962,-4.194173,4.916539,-2.035506,-24.422781,-20.989893,0,shadow_5


In [79]:
X = preds_df.drop(['membership', 'model'], axis = 1)
y = preds_df['membership']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

attack_model = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_leaf = 3)

attack_model.fit(X_train, y_train)
pred_train = attack_model.predict(X_train)
pred_test = attack_model.predict(X_test)

acc_train = accuracy_score(y_train, pred_train)
acc_test = accuracy_score(y_test, pred_test)

print("Accuracy Train:", acc_train)
print("Accuracy Test:", acc_test)

Accuracy Train: 0.9172
Accuracy Test: 0.76664
